In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import pickle
from pathlib import Path
from transformers import GPT2TokenizerFast

In [2]:
# need to had been set to root of all code repos, like '/home/toolkit/code/'
os.environ['PYTHONPATH']

'/home/toolkit/code/'

In [3]:
from repo_training_codellm.repo_context_prep.generate_hole_and_repo_contexts import getAllRulesContexts 

In [12]:
base_dir = Path('/repo_data/repo_preprocessed_data/')
data_split = 'train'
repo_name = 'phongcui1510'
rule_context_len = 2000

In [8]:
input_data_dir = base_dir / data_split / repo_name

#get stored parsed data
parsed_data_filename = input_data_dir / 'parsed_data'
with parsed_data_filename.open('rb') as f:
    parse_data = pickle.load(f)
    

#get the holes
hole_filename = input_data_dir / 'hole_data'
with hole_filename.open('rb') as f:
    hole_data = pickle.load(f)


In [9]:
files = list(input_data_dir.rglob('*.java'))

In [13]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
all_rule_context_obj = getAllRulesContexts(
    parse_data=parse_data,
    tokenizer=tokenizer,
    rule_context_len=rule_context_len
)

In [23]:
file = next(iter(hole_data.keys()))

In [29]:
l, c = hole_data[file][0]
hole_pos = l, c
hole_context, target_hole = all_rule_context_obj.get_hole_context(file, hole_pos)

In [31]:
hole_context

'\nimport com.icommerce.product.service.shared.Constants;\nimport org.springframework.context.annotation.Configuration;\nimport org.springframework.security.config.annotation.web.builders.HttpSecurity;\nimport org.springframework.security.config.annotation.web.builders.WebSecurity;\n package com.icommerce.'

In [32]:
target_hole

'product.configuration;\n'

In [33]:
rule_contexts, rule_context_locations = all_rule_context_obj.get_rule_contexts(file, hole_pos)

In [36]:
len(rule_contexts)

63

In [37]:
rule_context_locations

['in_file',
 'in_file',
 'in_file',
 'in_file',
 'in_file',
 'in_file',
 'in_file',
 'in_file',
 'parent_class_file',
 'parent_class_file',
 'parent_class_file',
 'parent_class_file',
 'parent_class_file',
 'parent_class_file',
 'import_file',
 'import_file',
 'import_file',
 'import_file',
 'import_file',
 'import_file',
 'sibling_file',
 'sibling_file',
 'sibling_file',
 'sibling_file',
 'sibling_file',
 'sibling_file',
 'similar_name_file',
 'similar_name_file',
 'similar_name_file',
 'similar_name_file',
 'similar_name_file',
 'similar_name_file',
 'child_class_file',
 'child_class_file',
 'child_class_file',
 'child_class_file',
 'child_class_file',
 'child_class_file',
 'import_of_sibling_file',
 'import_of_sibling_file',
 'import_of_sibling_file',
 'import_of_sibling_file',
 'import_of_sibling_file',
 'import_of_sibling_file',
 'import_of_similar_name_file',
 'import_of_similar_name_file',
 'import_of_similar_name_file',
 'import_of_similar_name_file',
 'import_of_similar_name_f

## Explore generated samples (holes)

In [ ]:
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
files = list(Path('/repo_data/repo_preprocessed_data/train/').glob('*/*.json'))
data = []
for file in tqdm(files):
    with file.open('rt') as f:
        while True:
            try:
                line = f.readline()
                if line is None or len(line) == 0:
                    break
                data.append(json.loads(line))
            except Exception:
                break

In [ ]:
stats_ctxs_len = [len(el['text']) for cdata in data for el in cdata['ctxs']]

In [ ]:
hist, bins = np.histogram(stats_ctxs_len, [0, 1, 10, 100, 1000, 10000, 100000, 1000000])

In [ ]:
s = sum(hist[1:])
for i in range (len(hist)):
    if i == 0: continue
    print(f'count: {hist[i]},  percentage: {(hist[i]/s*100):.02f}%, text length range chars: [{bins[i]}:{bins[i+1]-1}]')

In [ ]:
bins_ctx_lenght = bins[1:]
hist_ctx_length = hist[1:]
ress = plt.hist(bins_ctx_lenght[:-1], bins_ctx_lenght, weights=hist_ctx_length)
plt.xscale('log')
plt.title('Lenght of non empty contexts')

In [ ]:
stats_ctxs_count = [sum(len(el['text']) != 0 for el in cdata['ctxs']) for cdata in data]

In [ ]:
hist, bins = np.histogram(stats_ctxs_count, [0, 1, 10, 20, 30, 40, 50, 64])

In [ ]:
s = sum(hist[1:])
for i in range (len(hist)):
    if i == 0: continue
    print(f'count: {hist[i]},  percentage: {(hist[i]/s*100):.02f}%, count range: [{bins[i]}:{bins[i+1]-1}]')

In [ ]:
ress = plt.hist(bins[:-1], bins, weights=hist)
plt.title('Number of non empty contexts per sample')